In [ ]:
#1. Bibliotheken importieren
import mysql.connector as mysql
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly import tools
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
"""cursor.execute("SHOW DATABASES")

for x in cursor:
  print(x)"""

In [ ]:
#Read all the credentials from external file
credentials = pd.read_csv('../credentials/credentials.csv')

database_adress = credentials['Host'].iloc[0]
database_user = credentials['User'].iloc[0]
database_password = credentials['Password'].iloc[0]
database_name = credentials['Database'].iloc[0]

print(database_user)

In [ ]:
#Connect to Database
db = mysql.connect(host=database_adress, user=database_user, passwd=database_password, database=database_name)
cursor = db.cursor()

#Form and execute query
query = "SELECT * FROM movies;" ## Read from DB
cursor.execute(query)

#Load query in Dataframe and print it out
all_movies = pd.DataFrame(list(cursor), columns=['id', 'title', 'year'])

all_movies.iloc[0:10].style.hide_index()


In [ ]:
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

In [ ]:
db_connection_str = f"mysql+pymysql://{database_user}:{database_password}@{database_adress}/{database_name}"
db_connection = create_engine(db_connection_str)

In [ ]:
all_movies = pd.read_sql('SELECT * FROM movies ORDER BY release_year', con=db_connection)

In [ ]:
all_movies.head(400)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

year = pd.DataFrame(all_movies['release_year'].drop_duplicates())
count = all_movies.groupby(['release_year'])['movieId'].count()

year_count = pd.merge(year, count, on=["release_year", "release_year"])
year_count.rename(columns = {'movieId':'count'}, inplace=True)

year_count.drop(0, inplace=True)
year_count.drop(year_count.index[len(year_count)-1], inplace=True)

ax.bar(year_count['release_year'], year_count['count'], color ='maroon',
      width = 0.4)

ax.set_title("Released movies over time")
#ax.set_xlim(1850,2018)

In [ ]:
year_count['release_year'].head()

In [ ]:
all_movies.groupby(['release_year']).head()#['movieId'].count()

In [ ]:
count = all_movies.groupby(['release_year'])['movieId'].count()

test_fig, test_ax = plt.subplots(figsize=(10,5))

count = count.drop([-1])
count = count.drop([2018])

test_ax.bar(count.index, count.values, color ='maroon')

test_ax.set_title("Released movies over time")

In [ ]:
all_movies.groupby(['release_year']).head()

In [ ]:
movies_with_genres = pd.read_sql("""SELECT m.movieId, m.title, g.genrename FROM movies as m 
INNER JOIN movies_genres as mg ON m.movieID = mg.movieId
INNER JOIN genres as g ON mg.genreId = g.genreId
ORDER BY m.movieId;""", con=db_connection)

In [ ]:
genres = pd.read_sql("""SELECT * FROM genres;""", con=db_connection)

In [ ]:
genres_count = movies_with_genres.groupby(['genrename'])['movieId'].count()

genres_count_fig, genres_count_ax = plt.subplots(figsize=(30,15))
#genres_count.head()
genres_count_ax.bar(genres_count.index, genres_count.values, color ='maroon')

In [ ]:
genres.count()

In [ ]:
genres_df = pd.read_sql('SELECT mg.movieId,mg.genreId,g.genrename FROM movies_genres AS mg INNER JOIN genres AS g ON mg.genreId=g.genreId;', con=db_connection)

In [ ]:
genres_df.groupby(['genrename'])['movieId'].count()

## (1) Released movies over time (in Plotly)

In [ ]:
all_movies = pd.read_sql('SELECT * FROM movies ORDER BY release_year', con=db_connection)

In [ ]:
count = all_movies.groupby(['release_year'])['movieId'].count()

noyear_count = count[-1]
print(f'Movies without year: {noyear_count} or {((noyear_count / count.values.sum())*100):.2f}%')

count = count.drop([-1])
count = count.drop([2018])

count_df = count.to_frame()

fig1 = px.bar(count_df, x=count_df.index, y='movieId', title='Released movies over time',
labels={
                     "release_year": "Year of release",
                     "movieId": "Amount of movies"
                 },
                )
            
fig1.update_layout(
    xaxis=dict(
        title='Year of release',
        titlefont_size=16
    ),
    yaxis=dict(
        title='Amount of movies',
        titlefont_size=16
    )
)
fig1.update_traces(marker_color='maroon')
fig1.show()

## (3) Genre share over time

Diagram Type: 100% Stacked Area Chart (https://codejock.com/products/chart/100-stacked-area-chart.asp)

**Challenges**
* [OK] Multiple genres
* [OK] Amount of genres to display

In [ ]:
movies_genres = pd.read_sql('SELECT m.movieId, m.title, m.release_year, g.genreId, g.genrename FROM movies as m inner join movies_genres as mg on m.movieId=mg.movieId inner join genres as g on mg.genreId=g.genreId order by m.movieId', con=db_connection)
movies_genres[:10]

In [ ]:
print(f'Average genres per film: {len(movies_genres) / len(movies_genres.movieId.unique()):.2f}')
print('\nCheck total distribution')
genres_total_dist = movies_genres.genrename.value_counts().to_frame()
genres_total_dist['share'] = genres_total_dist.apply(lambda x: (x / genres_total_dist.genrename.sum()*100))
print(genres_total_dist)

In [ ]:
#Calculating right share based on amount of genres
#For example: Jumanji (1995) has 3 genres assigned. Every genre should only count 1/3. 

#DISCLAIMER: can take up to 5min! not optimized yet!

genres_hist = pd.DataFrame(index=movies_genres.genrename.unique())

for year in range(1900,2018+1):
    q = movies_genres.query(f'release_year == {year}').copy()
    q['genre_vote'] = q['movieId'].apply(lambda x: (1/len(q.query(f'movieId == {x}'))))
    q = q.groupby(by='genrename').sum()
    q = q['genre_vote']
    q = q.round(decimals=2)
    genres_hist[year] = q


#old approach: absolute amount genre per year
"""
for year in range(1900,2018+1):
    genres_hist[year] = movies_genres.query(f'release_year == {year}').genrename.value_counts()
"""

In [ ]:
#Fill na with 0 and set type int
genres_hist.fillna(0,inplace=True)
genres_hist = genres_hist.astype(int)

#calculate relative share for every year
for year in range(1900,2018+1):
    sum = genres_hist[year].sum()
    genres_hist[year] = genres_hist[year].apply(lambda x: round((x / sum)*100,2))

#Transpose for easier data access
genres_hist = genres_hist.T
genres_hist

In [ ]:
pio.templates.default = "plotly_dark"

top10genres = genres_hist.loc[2018].sort_values(ascending=False).head(10).index.to_list()

color_seq = px.colors.qualitative.G10
color_seq_count = 0

fig3 = go.Figure()
for genre in top10genres:
    color = color_seq[color_seq_count]
    color_seq_count+=1
    fig3.add_trace(go.Scatter(x=genres_hist.index,
                         y=genres_hist[genre],
                         #fill='tozeroy', #fill down to xaxis
                         #fillcolor='orange',
                         mode='lines',
                         line={'dash': 'solid', 'color': color},
                         name=genre,
                         stackgroup='one',
                         ))
fig3.update_layout(
    xaxis=dict(
        title='Time in years',
        titlefont_size=16
    ),
    yaxis=dict(
        title='Share of genre [%]',
        titlefont_size=16
    ),
    yaxis_range=(0,100),
    xaxis_range=(1900,2018),
    title="Genre popularity over time"
)
fig3.add_vrect(x0=1900,x1=1920,line_width=0,fillcolor='black',opacity=0.4,annotation_text="not enough data",annotation_position="bottom left",
    annotation=dict(font_size=18,font_color='white')
)
fig3.show()

## (4) Find potentially suspect activity (bots)

In [ ]:
users_with_one_rating = pd.read_sql('SELECT * FROM ratings as r WHERE (SELECT COUNT(*) FROM ratings as ra WHERE r.userId = ra.userId) = 1', con=db_connection)

In [ ]:
all_ratings = pd.read_sql('SELECT * FROM ratings', con=db_connection)

In [ ]:
users_with_one_rating.head()

In [ ]:
all_ratings.head()

In [ ]:
len(users_with_one_rating)

In [ ]:
len(all_ratings)

In [ ]:
users_with_one_rating.sort_values(by=['movieId']).to_csv('users_with_only_one_rating.csv', index=False)

In [ ]:
users_with_one_rating = pd.read_csv('users_with_only_one_rating.csv')

In [ ]:
users_with_one_rating.head()

In [ ]:
len(users_with_one_rating)

In [ ]:
previous_movie = 0
movie_ratings = 0
movies_and_rating_amount = []

for index, row in users_with_one_rating.iterrows():
    if row['movieId'] == previous_movie:
        movie_ratings += 1
        if len(users_with_one_rating) - 1 == index:
            movies_and_rating_amount.append([int(previous_movie), int(movie_ratings)])
    else:
        movies_and_rating_amount.append([int(previous_movie), int(movie_ratings)])
        previous_movie = row['movieId']
        movie_ratings = 1

movies_ratings_from_user_with_only_one_rating = pd.DataFrame(movies_and_rating_amount, columns=['Movie Id', 'Amount'])

In [ ]:
movies_ratings_from_user_with_only_one_rating.sort_values(by=['Amount'], ascending=False, inplace=True)
movies_ratings_from_user_with_only_one_rating.to_csv('movies_with_sus_ratings.csv', index=False)

In [ ]:
#Append movie title to dataframe

#Get relationship table between movieId and movie title
movies_ids_names = pd.read_sql('SELECT movieId, title FROM movies', con=db_connection)

#Drop last 0 entry to prevent error
movies_ratings_from_user_with_only_one_rating.drop(index=0, inplace=True, errors='ignore')

movies_ratings_from_user_with_only_one_rating['Movie Name'] = movies_ratings_from_user_with_only_one_rating['Movie Id'].apply(lambda x: movies_ids_names.query(f'movieId == {x}').title.values[0])
movies_ratings_from_user_with_only_one_rating.head(3)

In [ ]:
#pio.templates.default = "plotly_dark"

#iterate through movies with sus ratings to plot them (over time)
for i in movies_ratings_from_user_with_only_one_rating['Movie Id'].head(10):
    sus_rating = users_with_one_rating.query(f'movieId == {i}').copy()
    sus_rating.sort_values(by=['rating_date'], inplace=True)
    sus_rating_fig = px.scatter(sus_rating, x='rating_date', y='rating', title=f'Rating of movie {i} over time', range_x=['1998-01-01', '2019-01-01'], hover_data=['userId'])

    
    
    sus_rating_fig.show()
